# Voxel51 <-> V7 Multi-Slotted Walkthrough

## Load & prepare the data

In [5]:
from uuid import uuid4

import fiftyone as fo
import fiftyone.zoo as foz
import darwin_fiftyone

In [ ]:
# Load a subset of a grouped open Voxel51 dataset
group_dataset = foz.load_zoo_dataset("quickstart-groups", max_samples=10)
group_view = group_dataset.select_groups(group_dataset)
groups = group_view.select_group_slices(media_type="image")


To annotate multi-slotted dataset items, the relevant Voxel51 dataset must be a [grouped dataset](https://www.notion.so/Voxel51-Integration-fd9ee3a516714b3e93ddac4ef0ffbf0a?pvs=21). 

When working with grouped datasets, it’s required that a single `media_type` is specified for the `Dataset` or `View` by using the `select_group_slices()` function. This is because the `annotate()` function doesn’t directly support grouped datasets

In [ ]:

# Display the data in the Voxel51 app
session = fo.launch_app(groups)

# Define a label schema (Essentially an ontology of classes, see below for detail)
label_schema = {
    "new_detections": {
        "type": "detections",
        "classes": ["apple","orange"],
    }
}


## Upload the data to Darwin

In [ ]:
# Create a unique identifier for this annotation run
anno_key = f"key_{str(uuid4()).replace('-', '_')}"

groups.annotate(
    anno_key,
    label_schema = label_schema,
    launch_editor=True,
    backend="darwin",
    dataset_slug="group-test-dataset",
    base_url="https://darwin.v7labs.com/api/v2/teams",
    Groups=True,
)

The above method is uploading data stored locally (i.e. the `quickstart-groups` dataset, which is [already configured as a grouped dataset](https://docs.voxel51.com/user_guide/groups.html#quickstart-groups). So instead of having to assign samples to specific slots, it's already done for you when the dataset is loaded from the V51 side )

## Annotation in Darwin take place at this stage

In [ ]:
# After annotating, send the annotations back:
groups.load_annotations(anno_key)

# Finally, visualise the results back in Voxel51
session = fo.launch_app(groups)

In [ ]:
# To delete the associated V7 dataset and workflow you can use the cleanupmethod. It should be used only when you are finished with that data in V7.
results = groups.load_annotation_results(anno_key)
results.cleanup()